In [2]:
pip install eli5

     |████████████████████████████████| 106 kB 922 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [11]:
from typing import List, Dict

def get_labels(path: str) -> List[str]:
    with open(path, "r") as f:
        labels = f.read().splitlines()
        labels = [i if i != 'O' else 'O' for i in labels]
    if "O" not in labels:
        labels = ["O"] + labels
    return labels


In [52]:
from transformers import (
    AutoConfig,
    AutoModelForTokenClassification,
    AutoTokenizer)


# MODEL_NAME_OR_PATH =  'dmis-lab/biobert-base-cased-v1.1'
MODEL_NAME_OR_PATH =  '../../models/word_embeddings/fine_tuned/NER/ACD_15epochs'

config = AutoConfig.from_pretrained(MODEL_NAME_OR_PATH)

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME_OR_PATH,
    do_lower_case=False
)
model = AutoModelForTokenClassification.from_pretrained(
    MODEL_NAME_OR_PATH,
    config=config,
)

labels = [value for k, value in config.id2label.items()]
# print(config)

In [45]:
import torch 
import numpy as np

test_sentence = 'In two of the three deaths probably associated with ketoconazole treatment the drug had been continued after the onset of jaundice and other symptoms of hepatitis.'

tokenized_sentence = tokenizer.encode(test_sentence)
input_ids = torch.tensor([tokenized_sentence])

with torch.no_grad():
    output = model(input_ids)

label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)

tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])
new_tokens, new_labels = [], []

for token, label_idx in zip(tokens, label_indices[0]):
    if token.startswith("##"):
        new_tokens[-1] = new_tokens[-1] + token[2:]
    else:
        new_labels.append(labels[label_idx])
        new_tokens.append(token)
for token, label in zip(new_tokens, new_labels):
    print("{}\t\t{}".format(token, label))

[CLS]		O
In		O
two		O
of		O
the		O
three		O
deaths		B-Disease
probably		O
associated		O
with		O
ketoconazole		B-Chemical
treatment		O
the		O
drug		O
had		O
been		O
continued		O
after		O
the		O
onset		O
of		O
jaundice		B-Disease
and		O
other		O
symptoms		O
of		O
hepatitis		B-Disease
.		O
[SEP]		O


In [47]:
from eli5.lime import TextExplainer
from eli5.lime.samplers import MaskingTextSampler

sampler = MaskingTextSampler(
    replacement="UNK",
    max_replace=0.7,
    token_pattern=None,
    bow=False
)

samples, similarity = sampler.sample_near(test_sentence, n_samples=4)
print(test_sentence)
print(samples)

In two of the three deaths probably associated with ketoconazole treatment the drug had been continued after the onset of jaundice and other symptoms of hepatitis.
('In two of the UNK deaths probably associated with ketoconazole treatment the drug had been UNK after UNK onset UNK jaundice UNK other UNK of hepatitis.', 'In two UNK UNK three deaths UNK associated UNK ketoconazole treatment UNK UNK UNK been UNK UNK UNK onset of UNK and other symptoms of hepatitis.', 'In UNK of the UNK deaths UNK UNK with UNK treatment the drug UNK UNK UNK UNK the UNK of UNK UNK other UNK UNK hepatitis.', 'In two of the three deaths probably associated with ketoconazole UNK the drug had been continued after the onset of jaundice and other symptoms of hepatitis.')


In [127]:
class NERExplainerGenerator(object):
    
    def __init__(self, model):
        self.model = model
       
        
#     def _preprocess(self, texts):
#         X = [[self.word2idx.get(w, self.word2idx["UNK"]) for w in t.split()]
#              for t in texts]
#         X = pad_sequences(maxlen=self.max_len, sequences=X,
#                           padding="post", value=self.word2idx["PAD"])
#         return X
    
    def get_predict_function(self, word_index):
        def predict_func(input_ids):
#             X = self._preprocess(texts)
            print('aqui')
#             print(input_ids[0][:])
            p = self.model(input_ids)
            p_array = p['logits'][0][word_index]
            print(type(p))
            print(type(p['logits']))

            an_array = p_array.eval(session=tf.compat.v1.Session())
            print('p_array ', an_array)
            return p['logits'][0][word_index]
        return predict_func(input_ids)

explainer_generator = NERExplainerGenerator(model)
word_index = 4

predict_func = explainer_generator.get_predict_function(word_index=word_index)
predict_func
te = TextExplainer(
    sampler=sampler,
    position_dependent=True,
    random_state=42
)

te.fit(test_sentence, predict_func)

# te.explain_prediction(
#     target_names=list(explainer_generator.idx2tag.values()),
#     top_targets=3
# )

aqui
<class 'transformers.modeling_outputs.TokenClassifierOutput'>
<class 'torch.Tensor'>


AttributeError: 'Tensor' object has no attribute 'eval'